![header](http://eurogoos.eu/wp-content/uploads/SOCIB-logo.jpg)

# SOCIB API TRAINING
<div style="text-align: right"><i> 04-Part-one-out-of-04 </i></div>

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item">
    <li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">
        <span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span>
    </li>
    <li><span><a href="#Setup" data-toc-modified-id="Setup-2">
        <span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span>
        <ul class="toc-item">
            <li>
                <span><a href="#Importing-modules" data-toc-modified-id="Importing-modules-2.1">
                    <span class="toc-item-num">2.1&nbsp;&nbsp;</span>Importing modules</a></span>
            </li>
            <li>
                <span><a href="#API-token" data-toc-modified-id="API-token-2.2">
                    <span class="toc-item-num">2.2&nbsp;&nbsp;</span>API token</a></span>
            </li>
        </ul>
    <li>
        <span><a href="#Entry-data-as-json" data-toc-modified-id="Entry-data-as-json"><span class="toc-item-num">3&nbsp;&nbsp;</span>Entry data as json:</a></span>
        <!--<ul>
            <li><span><a href="#Generic-query" data-toc-modified-id="Generic-query"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Generic query</a></span></li>
            <li><span><a href="#Custom-query" data-toc-modified-id="Custom-query"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Custom query</a></span></li>
            <ul>
            <li><span><a href="#Subsetting-operations" data-toc-modified-id="Subsetting-operations"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Subsetting operations</a></span></li>
            <li><span><a href="#Resampling-operations" data-toc-modified-id="Resampling-operatios"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Resampling operations</a></span></li>
            </ul>-->
        </ul>
     </li>
    </ul>
</div>

---

# ENTRIES

## Introduction 

SOCIB API is a door users can knock-on in order to get information about the Balearic Islands Coastal Ocean Observing and Forecast System (SOCIB). SOCIB API is represented by an generic url (SOCIB API url). The elements that trigger a response when added to the generic API url are called `ENDPOINTS`.
Among the present posibilities:
<ul>
    <li>measured variables (<span class="alert-info">/standard-variables/</span>)</li>
    <li>stock of instruments (<span class="alert-info">/instrument-types/</span>) and platforms(<span class="alert-info">/platform-types/</span>), 
</li>
    <li>data maturity (<span class="alert-info">/processing-levels/</span> and <span class="alert-info">/data-modes/</span>)</li>
    <li>kind of data (<span class="alert-info">/feature-types/</span>)</li>
    <li>data entities (<span class="alert-success"><b>/entries/<b></span>, <span class="alert-info">/data-sources/</span>, <span class="alert-info">/platforms/</span>, <span class="alert-info">/data-products/</span>)
    </li>

</ul>

<br>This notebook will focus then on the <span class="alert-success" style=""><b>/entries/</b></span> endpoint.

---

## Setup

### Importing modules

We will relly on a set of python modules to deal with SOCIB API next.<br> `Please run the next cell` so that they can be used by the present Jupyter Notebook:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import datetime
import json
import requests
import IPython
import pandas
from json2html import *
%matplotlib inline

<div class="alert alert-block alert-warning" style="margin-left: 2em">
<b>Tip!</b>
    
***  
If any of them raises any error please run prior to load it and in a dedicated cell, either:<ul><li> <i>`!conda install packageName --yes`</i></li> or <li><i>`!pip install packageName --yes`</i></li></ul>

### API token

To be able to query SOCIB API you will need first a <i>token</i> (api key).<br>To get one please visit the [API home page](http://api.socib.es/home/) and fill-in the form at bottom. An email will be sent to you with such <i>token</i>.

`Please run the next cell if you wanna have a glimpse to API home page`:

In [ ]:
IPython.display.HTML('<iframe src="http://api.socib.es/home/" width=100% height=500></iframe>')

`Please set in the next cell your api_key and run the cell below to load it in memory for later use`:

In [ ]:
api_key = '' #write here the token emailed to you

In [ ]:
api_url = 'http://api.socib.es'
headers = {
    'accept': 'application/vnd.socib+json',
    'apikey': api_key,
}

<div class="alert alert-block alert-warning" style="margin-left: 2em">
<b>Tip!</b>
    
***  
If you do not remember your token or wanna ask for a new one please reach <i>data.center@socib.es</i> with the following `subject`: 'SOCIB API TOKEN: UPDATE/REMIND REQUEST'

---

## Entry data as json

As we akready saw in [Getting Started]('../01-GettingStarted.ipynb'), the basic metocean entity are netCDF files, as this is the chosen format to store and distribute the data reported by instrument & platforms enssembles. Also, we have already seen on [03-entry-services]('../03-entry-services.ipynb') that SOCIB provides a set of services for users to download and access such entries/files. i.e: <i>CTD data form the vessel "Garcia del Cid"</i>

In [ ]:
end_point = '/entries/'
query_parameters = '?platform_type=Oceanographic Buoy'
url = '%s%s%s' % (api_url, end_point, query_parameters)
entries_request = requests.get(url, headers=headers)
entries_response = json.loads(entries_request.text)

print('API query url: '+ url)
print('Services for the first entry/file on list (id=%s) :'%(entries_response['results'][2]['id']))
IPython.display.HTML(json2html.convert(json=entries_response['results'][0]))

Nevertheless, all these services pivot around a certain file format: the netCDF. The netCDF format is widely used and known among the MetOcena community but not so much beyond. Do I have to master in reading netCDF files and other possible formats to finally retrieve the observations they are stoing?. <br>
SOCIB API acts as a layer of abstraction between users and the underlying file format storing the observations trough the  `/data/` auxiliary endpoint. Let's se how it works!

Let's request on day of data from one of the above variable in the entry/file using the auxiliary endpoint:

In [ ]:
ini = entries_response['results'][0]['initial_datetime']
ini

In [ ]:
end = datetime.datetime.strftime(datetime.datetime.strptime(entries_response['results'][0]['initial_datetime'], '%Y-%m-%dT%H:%M:%SZ') + datetime.timedelta(days=1),'%Y-%m-%dT%H:%M:%SZ')
end

In [ ]:
var = 'sea_water_temperature'

In [ ]:
entry_id = entries_response['results'][0]['id']
endpoint = '/entries/'
aux_endpoint = '/data/'
query_parameters = '?standard_variable=%s&initial_datetime=%s&end_datetime=%s'%(var,ini,end)
url = '%s%s%s%s%s' % (api_url, end_point, entry_id, aux_endpoint, query_parameters)
entry_request = requests.get(url, headers=headers)
entry_response = json.loads(entry_request.text)

print('API query url: '+ url)
print('Data of the entry above (id=%s): '%(entry_id))
IPython.display.HTML(json2html.convert(json=entry_response))

---

## Next tutorial

<div class="alert alert-block alert-success" style="margin-left: 2em">
<b>More!</b>
    
***  
To see way more about SOCIB API, check next the `data-sources` dedicated notebooks:
<ul>
    <li><span><a href="../data-sources/01-data-source-details.ipynb">01-data-source-details</a></span></li>
    <li><span><a href="../data-sources/02-data-source-search.ipynb">02-data-source-search</a></span></li>
    <li><span><a href="../data-sources/03-data-source-composing-entries.ipynb">03-data-source-composing-entries</a></span></li>
    <li><span><a href="../data-sources/04-data-source-data-as-json.ipynb">04-data-source-data-as-json</a></span></li>
</ul>